#**Pre-request**

##env


In [10]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Navigate to project location
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
configs/  notebooks/  requirements-lock.txt  results/           src/
dataset/  README.md   requirements.txt       run_experiment.py  tests/


##Install pakages


In [11]:
#Install pakages
%pip install -r requirements.txt --no-cache-dir
######Restart sesstion after this line at first run to make sure that the new libs are used====

##Import  libs

In [12]:
%pip freeze > requirements-lock.txt
import os
import pandas as pd
import yaml
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from ydata_profiling import ProfileReport
import sweetviz as sv
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import datetime



#Helper Code

##Loging

In [13]:
# Make sure results directory exists
os.makedirs("results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("results/data_prep.log")
    ]
)
logger = logging.getLogger(__name__)



##Config

In [14]:
def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file"""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    logger.info(f"Loaded config from {config_path}")
    return config

## CDR dataset

In [15]:
def load_cdr(file_path, nrows=None):
    """Load a CDR CSV file with optional row limit (sample mode)."""
    logger.info(f"Loading file: {file_path} with nrows={nrows}")
    return pd.read_csv(file_path, nrows=nrows)


def load_all_data(config):
    """
    Load all CSVs defined in config into a dict of DataFrames.
    Uses training.sample_size if available.
    """
    base = config["dataset"]["base_path"]
    files = config["dataset"]["files"]
    sample_size = config.get("training", {}).get("sample_size", None)

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path, nrows=sample_size)
        data[name] = df
        logger.info(f"Loaded {name} -> {df.shape} from {path}")
    return data


##Genrate profile

In [16]:
import os

def ydata_generate_profile(df, output_file="results/profile.html", mode="minimal", overwrite=True):
    """
    Generate profiling report (ydata-profiling) with safe defaults.
    """
    # Drop very high-cardinality or long-text columns in minimal mode
    if mode == "minimal":
        drop_cols = [
            col for col in df.columns
            if df[col].nunique() > 500 or df[col].astype(str).str.len().mean() > 50
        ]
        if drop_cols:
            print(f"⚠️ Skipping columns for profiling: {drop_cols}")
            df = df.drop(columns=drop_cols)

    # Handle overwrite manually
    if overwrite and os.path.exists(output_file):
        os.remove(output_file)

    profile = ProfileReport(
        df,
        title="Fraud Detection EDA Report",
        explorative=True,
        plot={"wordcloud": False},
        correlations={
            "pearson": {"calculate": mode == "full"},
            "spearman": {"calculate": mode == "full"},
            "kendall": {"calculate": mode == "full"},
            "phi_k": {"calculate": mode == "full"},
            "cramers": {"calculate": mode == "full"},
            "auto": {"calculate": mode == "full"},
        },
        interactions={"continuous": mode == "full"},
        missing_diagrams={"heatmap": mode == "full"},
        duplicates={"calculate": mode == "full"}
    )

    profile.to_file(output_file)  # ✅ no overwrite argument
    print(f"✅ Profiling report saved → {output_file}")
    return output_file


In [17]:
def generate_profile_sweetviz(df, output_file="results/profile_sweetviz.html", target=None, overwrite=True):
    """
    Generate a Sweetviz report for df (with optional target column).
    """
    import os
    if overwrite and os.path.exists(output_file):
        os.remove(output_file)

    # avoid extremely high cardinality columns (optional)
    drop_cols = [col for col in df.columns if df[col].nunique() > 1000]
    if drop_cols:
        print(f"⚠️ Dropping high-cardinality columns for Sweetviz: {drop_cols}")
        df = df.drop(columns=drop_cols)

    report = sv.analyze(df, target_feat=target)
    report.show_html(output_file)   # ✅ now matches ydata `to_file()`
    print(f"✅ Profiling report saved → {output_file}")
    return output_file



# Load config +  dataset
# ===============================


In [18]:
config = load_config("configs/baseline.yaml")

# Load raw data
data = load_all_data(config)


for name, df in data.items():
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # Profiling with ydata_profiling
    profiling_mode = config["training"].get("profiling_mode", "minimal")
    ydata_output = f"results/profile_{name}_{timestamp}.html"
    ydata_generate_profile(df, output_file=ydata_output, mode=profiling_mode)

    # Profiling with Sweetviz
    target = "label" if "label" in df.columns else None
    sweetviz_output = f"results/profile_{name}_sweetviz_{timestamp}.html"
    generate_profile_sweetviz(df, output_file=sweetviz_output, target=target)




⚠️ Skipping columns for profiling: ['phone_no_m', 'busi_name', 'flow']


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 14.11it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Profiling report saved → results/profile_voc_20250927_115650.html
⚠️ Dropping high-cardinality columns for Sweetviz: ['opposite_no_m', 'start_datetime']


                                             |          | [  0%]   00:00 -> (? left)

Report results/profile_voc_sweetviz_20250927_115650.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
✅ Profiling report saved → results/profile_voc_sweetviz_20250927_115650.html
